In [ ]:
# Desenvolvimento de ACs para a simulação da propagação de FakeNews

# Importar as bibliotecas utilizadas no decorrer do programa:
from tomato.classes import cell
from matplotlib import pyplot as plt
import numpy as np

# Determinação das regras de transição:
class FakeNews(cell.CellTemplate):
    
    def update(self, state_matrix):
        
        self.state_matrix = state_matrix

        # Célula NEUTRAL:
        if self.value == 0:
            # Mais do que 4 vizinhos FAKE, se torna FAKE
            if self.neighbors_fake >= 4:
                self.value = -1
            
            # Mais do que 4 vizinhos TRUTH, se torna TRUTH
            elif self.neighbors_truth >= 4:
                    self.value = 1
            
            else:
                self.value = 0
                
        # Célula FAKE:   
        if self.value == -1:
            # Mais do que 7 vizinhos TRUTH, se torna TRUTH
            if self.neighbors_truth >= 7:
                self.value = 1
            
            # Entre 5 e 6 vizinhos TRUTH, se torna NEUTRAL
            elif (self.neighbors_truth == 5) or (self.neighbors_truth == 6):
                    self.value = 0
                    
            else:
                self.value = -1
                
        # Célula TRUTH:
        if self.value == 1:
            # Mais do que 7 vizinhos FAKE, se torna FAKE
            if self.neighbors_fake >= 7:
                self.value = -1
            
            # Entre 5 e 6 vizinhos FAKE, se torna NEUTRAL
            elif (self.neighbors_fake == 5) or (self.neighbors_fake == 6):
                self.value = 0
                    
            else:
                self.value = 1
    
    @property
    def neighbors(self):
        return self.moore_neighborhood
        # Considera-se a vizinhança de MOORE

    @property
    def neighbors_fake(self):
        return self.neighbors.count(-1)
        # Conta o n° de vizinhos FAKE

    @property
    def neighbors_neutral(self):
        return self.neighbors.count(0)
        # Conta o n° de vizinhos NEUTRAL
        
    @property
    def neighbors_truth(self):
        return self.neighbors.count(1)
        # Conta o n° de vizinhos TRUTH   

    # Definição das cores de cada tipo de célula:
    @staticmethod
    def display(value):
        if value == -1:
            return (255,0, 0)
        # FAKE (vermelho)
        
        elif value == 0:
            return (255,255,255)
        # NEUTRAL (branco)
        
        else:
            return (0,255,0)
        # TRUTH (verde)

    @staticmethod
    def from_display(value):
        if (value == (255,0, 0)).all():
            return -1
        
        elif (value == (255,255,255)).all():
            return 0
        
        else:
            return 1

In [ ]:
import tomato as tt
from tomato.functions import utils

rule = FakeNews

CELL_SIZE = 40
dimensions = (100, 100)

state_matrix = np.random.choice(a=[-1, 0, 1], size = dimensions, p=[0.235, 0.235, 0.53])
initial_state_matrix = state_matrix

board = tt.Board(rule, cell_size=CELL_SIZE)
board.load_state(initial_state_matrix)

# Criando vetores numpy para armazenar as populações de cada espécie de célula
fake_pop = np.zeros(200)
neutral_pop = np.zeros(200)
truth_pop = np.zeros(200)

while board.generation < 200:
    # Matriz com os estados de cada célula:
    state_matrix = board.state_matrix

    fake_pop[board.generation] = np.count_nonzero(state_matrix == -1)
    neutral_pop[board.generation] = np.count_nonzero(state_matrix == 0)
    truth_pop[board.generation] = np.count_nonzero(state_matrix == 1)

    # Iterar a simulação depois de terminada a nossa coleta de dados da geração
    board.update()

# Avisando que a simulação acabou e fazendo umas estatísticas básicas
print("Prontinho!")
print(f"neutral_pop: initial {neutral_pop[0]} | final {neutral_pop[-1]} | avg {np.mean(neutral_pop)}")
print(f"truth_pop: initial {truth_pop[0]} | final {truth_pop[-1]} | avg {np.mean(truth_pop)}")
print(f"fake_pop: initial {fake_pop[0]} | final {fake_pop[-1]} | avg {np.mean(fake_pop)}")

initial_neutral = neutral_pop[0]/(neutral_pop[0] + truth_pop[0] + fake_pop[0])
initial_truth = truth_pop[0]/(neutral_pop[0] + truth_pop[0] + fake_pop[0])
initial_fake = fake_pop[0]/(neutral_pop[0] + truth_pop[0] + fake_pop[0])

final_neutral = neutral_pop[-1]/(neutral_pop[-1] + truth_pop[-1] + fake_pop[-1])
final_truth = truth_pop[-1]/(neutral_pop[-1] + truth_pop[-1] + fake_pop[-1])
final_fake = fake_pop[-1]/(neutral_pop[-1] + truth_pop[-1] + fake_pop[-1])



In [ ]:
fig, ax = plt.subplots()
ax.plot(neutral_pop, color="gray", label="Neutral")
ax.plot(truth_pop, color="green", label="Truth")
ax.plot(fake_pop, color="red", label="Fake")

ax.set_title("Evolução das populações")
ax.set_xlabel("Geração")
ax.set_ylabel("População")
ax.legend()

In [ ]:
initial_neutral = neutral_pop[0]/(neutral_pop[0] + truth_pop[0] + fake_pop[0])
initial_truth = truth_pop[0]/(neutral_pop[0] + truth_pop[0] + fake_pop[0])
initial_fake = fake_pop[0]/(neutral_pop[0] + truth_pop[0] + fake_pop[0])

final_neutral = neutral_pop[-1]/(neutral_pop[-1] + truth_pop[-1] + fake_pop[-1])
final_truth = truth_pop[-1]/(neutral_pop[-1] + truth_pop[-1] + fake_pop[-1])
final_fake = fake_pop[-1]/(neutral_pop[-1] + truth_pop[-1] + fake_pop[-1])

mycolors = ["gray", "green", "red"]
# mycolors = ["green", "red"]
mylabels = ["Neutral", "Truth", "Fake"]
# mylabels = ["Truth", "Fake"]
myexplode = [0.05, 0.05, 0.05]
# myexplode = [0.05, 0.05]

# Populações iniciais:
y = np.array([initial_neutral, initial_truth, initial_fake])
# y = np.array([initial_truth, initial_fake])
plt.title("Populações iniciais")
plt.pie(y, labels = mylabels, startangle = 90, explode = myexplode, shadow = True, autopct='%1.1f%%', colors = mycolors)
plt.show()

# Populações finais:
y = np.array([final_neutral, final_truth, final_fake])
# y = np.array([final_truth, final_fake])
plt.title("Populações finais")
plt.pie(y, labels = mylabels, startangle = 90, explode = myexplode, shadow = True, autopct='%1.1f%%', colors = mycolors)
plt.show() 

In [ ]:
import tomato as tt
from tomato.functions import utils

rule = FakeNews

CELL_SIZE = 40
dimensions = (100, 100)

state_matrix = np.random.choice(a=[-1, 0, 1], size = dimensions, p=[0.10, 0.80, 0.10])

board = tt.Board(rule, img_cell_size=CELL_SIZE)

board.start(
    state_matrix, 
    inline = True, 
    generations = 200, 
    generate_figures = True, 
    generate_figures_dir = "Trabalho_Final", # Especificar o nome da pasta
    generate_gif = True # Criar um gif com as imagens ao final da execução
)

board.save_png("Modelo_1_GIF.png")